In [3]:
import pandas as pd

df = pd.read_csv('../data/flights.csv')
df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-12-18,UA,UA,UA,831,UA,N453UA,831,11423,DSM,...,589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-07-07,UA,UA_CODESHARE,UA,5560,OO,N933EV,5560,11292,DEN,...,776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-04-06,AA,AA,AA,1689,AA,N102UW,1689,14492,RDU,...,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-07-15,UA,UA_CODESHARE,UA,5508,OO,N964SW,5508,12892,LAX,...,726,0.0,0.0,0.0,0.0,72.0,NaN,NaN,NaN,NaN
4,2019-05-19,WN,WN,WN,4793,WN,N8686A,4793,14683,SAT,...,1407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Preprocessing

In [4]:
df['weather_delay'] = df['weather_delay'].fillna(0)
df = df[['fl_date', 'dep_time', 'origin', 'dest', 'distance', 'weather_delay']]

In [12]:
# Assuming df is your DataFrame
# Separate the DataFrame into two groups
delayed_flights = df[df['weather_delay'] > 0]
non_delayed_flights = df[df['weather_delay'] == 0]

# Calculate the number of samples you need from each group
total_samples = 10000  # 20% of the total data
num_delayed_samples = int(total_samples * 0.10)  # 20% of the sample should have delay
num_non_delayed_samples = int(total_samples - num_delayed_samples)

# Sample from each group
delayed_sample = delayed_flights.sample(n=num_delayed_samples, random_state=42)
non_delayed_sample = non_delayed_flights.sample(n=num_non_delayed_samples, random_state=42)

# Combine the samples
sampled_df = pd.concat([delayed_sample, non_delayed_sample])

# Shuffle the sampled DataFrame (if desired)
df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [13]:
def get_lat_lon_from_iata(airport_codes):
    df = pd.read_csv('../data/iata-icao.csv')
    code_to_loc = {}
    for code in airport_codes:
        lat, lon = df[df['iata'] == code]['latitude'].iloc[0], df[df['iata'] == code]['longitude'].iloc[0]
        if code not in code_to_loc:
            code_to_loc[code] = (lat, lon)
    return code_to_loc

lat_lon_dict = get_lat_lon_from_iata(list(df['origin'].unique()))

In [14]:
df.dropna(axis=0, inplace=True)

In [15]:
from weather_data import fetch_weather_data
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np

def add_weather_data(row, lat_lon_dict):
    origin_lat_lon = lat_lon_dict[row['origin']]
    hours = int(row['dep_time'] / 100)
    minutes = int(row['dep_time'] % 100)
    if hours == 24:
        hours = 0
        
    datetime_str = f"{row['fl_date']}T{hours:02d}:{minutes:02d}:00"
    weather_origin = fetch_weather_data(*origin_lat_lon, datetime_str)
    return {**weather_origin}

def parallelize_dataframe(df, func, lat_lon_dict, n_jobs=8):
    # Split DataFrame
    df_split = np.array_split(df, n_jobs)
    
    # Process DataFrame in parallel
    results = []
    with ThreadPoolExecutor(max_workers=n_jobs) as executor:
        futures = {executor.submit(process_chunk, chunk, func, lat_lon_dict) for chunk in df_split}
        for future in as_completed(futures):
            results.append(future.result())

    # Concatenate results
    return pd.concat(results)

def process_chunk(chunk, func, lat_lon_dict):
    return chunk.apply(lambda row: func(row, lat_lon_dict), axis=1)

# Example usage
n_jobs = 4  # Number of parallel jobs
weather_columns = parallelize_dataframe(df, add_weather_data, lat_lon_dict, n_jobs)
weather_df = pd.DataFrame(list(weather_columns))

# Reset the index of both DataFrames to ensure proper alignment
df = pd.concat([df.reset_index(drop=True), weather_df.reset_index(drop=True)], axis=1)

# Display the first few rows
df.head()


/Users/pandita/Documents/witd/streamlit_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
Failed to retrieve weather data: 429
F

,fl_date,dep_time,origin,dest,distance,weather_delay,datetime,datetimeEpoch,temp,feelslike,...,visibility,cloudcover,solarradiation,solarenergy,uvindex,conditions,icon,stations,source,tzoffset
0,2019-07-02,705.0,CVG,DCA,411,0.0,18:00:00,1.544569e+09,38.0,31.6,...,9.9,0.0,2.0,0.0,0.0,Clear,clear-night,"[TT136, 72408013739, 72408594732, 72407493780,...",obs,NaN
1,2018-02-24,1724.0,LAS,DEN,628,0.0,21:00:00,1.554080e+09,48.8,44.1,...,9.9,88.2,0.0,0.0,0.0,Partially cloudy,partly-cloudy-night,"[KFTY, 72219013874, KATL, 72219503888, 7221965...",obs,NaN
2,2018-05-09,1157.0,MEM,ATL,332,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-06-26,1227.0,EWR,DTW,488,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-03,1720.0,PHL,ERI,300,0.0,20:00:00,1.544494e+09,45.0,45.0,...,9.9,27.1,517.0,1.9,0.0,Partially cloudy,partly-cloudy-night,"[72059400188, 72242953910, KIAH, KDWH, KMCJ, A...",obs,NaN


In [16]:
df.dropna(subset=['datetime'], inplace=True)
df.head()

,fl_date,dep_time,origin,dest,distance,weather_delay,datetime,datetimeEpoch,temp,feelslike,...,visibility,cloudcover,solarradiation,solarenergy,uvindex,conditions,icon,stations,source,tzoffset
0,2019-07-02,705.0,CVG,DCA,411,0.0,18:00:00,1.544569e+09,38.0,31.6,...,9.9,0.0,2.0,0.0,0.0,Clear,clear-night,"[TT136, 72408013739, 72408594732, 72407493780,...",obs,NaN
1,2018-02-24,1724.0,LAS,DEN,628,0.0,21:00:00,1.554080e+09,48.8,44.1,...,9.9,88.2,0.0,0.0,0.0,Partially cloudy,partly-cloudy-night,"[KFTY, 72219013874, KATL, 72219503888, 7221965...",obs,NaN
4,2019-01-03,1720.0,PHL,ERI,300,0.0,20:00:00,1.544494e+09,45.0,45.0,...,9.9,27.1,517.0,1.9,0.0,Partially cloudy,partly-cloudy-night,"[72059400188, 72242953910, KIAH, KDWH, KMCJ, A...",obs,NaN
5,2019-06-13,1417.0,MSP,CWA,175,0.0,05:00:00,1.563527e+09,76.9,76.9,...,9.9,25.6,0.0,0.0,0.0,Partially cloudy,partly-cloudy-night,"[99999913752, 72215899999, 72406093721, KBWI, ...",obs,NaN
6,2018-04-10,2238.0,FLL,BWI,925,44.0,16:00:00,1.562616e+09,81.0,80.8,...,9.9,59.1,720.0,2.6,7.0,Partially cloudy,partly-cloudy-day,"[72537094847, 72537614853, KONZ, KYIP, C4874, ...",obs,NaN


In [17]:
# from serpapi import GoogleSearch
# from datetime import datetime, timedelta

# def search_with_date_filter(query, start_date, end_date):
#     """
#     Perform a search using a SERP API with date filters.

#     Args:
#     query (str): The search query.
#     start_date (datetime.date): The start date for filtering results.
#     end_date (datetime.date): The end date for filtering results.
#     api_key (str): Your API key for the SERP API service.

#     Returns:
#     dict: The search results returned by the API.
#     """
#     api_key = "3be86335905bfda27c9bd8e81b6b1c7147d6df2369c0db6139688f2ed3d75344"
#     # Format dates in the required format, e.g., YYYY-MM-DD
#     start_date_str = start_date.strftime('%m/%d/%Y')
#     end_date_str = end_date.strftime('%m/%d/%Y')

#     # Define the parameters for the API request
#     params = {
#         'engine': 'google_news',
#         'api_key': api_key,
#         'q': query,
#         'tbs': f'cdr:1,cd_min:{start_date_str},cd_max:{end_date_str}'
#         # Add other parameters as required by your specific SERP API
#     }

#     # Make the API request
#     search = GoogleSearch(params)
#     results = search.get_dict()
#     if results and 'news_results' in results.keys(): 
#         #print([(start_date,datetime.strptime(result['date'].split(',')[0], '%m/%d/%Y').date(),end_date) for result in results["news_results"]])
#         #print([result for result in results["news_results"] if start_date <= datetime.strptime(result['date'].split(',')[0], '%m/%d/%Y').date() <= end_date])
#         return [result for result in results["news_results"] if start_date <= datetime.strptime(result['date'].split(',')[0], '%m/%d/%Y').date() <= end_date]
#     else:
#         return []

In [18]:
# import concurrent.futures
# from bs4 import BeautifulSoup as Soup
# from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
# import langchain_core

# def create_query(row):
#     return f'"{row["origin"].split(",")[0]}" + "{row["dest"].split(",")[0]}" + "delay"'
#     #return "JFK LAX flight delay"

# def remove_nav_and_header_elements(page):
#     content = Soup(page, 'html.parser')
#     exclude = content.find_all(["nav", "footer", "header", "head"])
#     for element in exclude:
#         element.decompose()

#     return str(content.text).strip()

# def fetch_articles(url):
#     try:
#         loader = RecursiveUrlLoader(
#             url=url, max_depth=1, extractor=remove_nav_and_header_elements
#         )
#         doc = loader.load()
#         return doc[0] if doc else ""

#     except Exception as e:
#         return str(e)

# def fetch_articles_for_query(row):
#     query = row['query']
#     end_date = datetime.strptime(row['fl_date'], '%Y-%m-%d').date()
#     #end_date = datetime.strptime('2023-06-27', '%Y-%m-%d').date()
#     start_date = end_date - timedelta(days=1)

#     results = search_with_date_filter(query, start_date, end_date)
#     urls = [x['link'] for x in results if 'pdf' not in x['link']]
#     articles = []
#     with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#         future_to_url = {executor.submit(fetch_articles, url): url for url in urls}
#         for future in concurrent.futures.as_completed(future_to_url):
#             article = future.result()
#             if type(article) == langchain_core.documents.base.Document:
#                 if not ('403 Forbidden' in article.metadata['title'] or 'Error' in article.metadata['title']):
#                     articles.append(article.page_content)
#     return articles

# df['query'] = df.apply(create_query, axis=1)
# df['articles'] = df.apply(fetch_articles_for_query, axis=1)
# # articles = fetch_articles_for_query(df.iloc[1])

In [19]:
# Convert 'fl_date' to datetime
df['fl_date'] = pd.to_datetime(df['fl_date'])
# Extract month and day as integers
df['month'] = df['fl_date'].dt.month
df['day'] = df['fl_date'].dt.day

df.dropna(subset=['datetime'], inplace=True)

df['hours'] = df['datetime'].apply(lambda x: int(x.split(':')[0]))
df['minutes'] = df['datetime'].apply(lambda x: int(x.split(':')[1]))

In [20]:
def list_to_str(value):
    if value is None:
        return ''  # or whatever you want to return when the value is None
    elif isinstance(value, list) and len(value) > 0:
        return str(value[0])  # or some other logic for turning a list into a string
    else:
        return str(value)  # convert non-list and empty list values to string

df['preciptype'] = df['preciptype'].apply(list_to_str)
df['preciptype'] = df['preciptype'].replace('nan', '')
df.head()

,fl_date,dep_time,origin,dest,distance,weather_delay,datetime,datetimeEpoch,temp,feelslike,...,uvindex,conditions,icon,stations,source,tzoffset,month,day,hours,minutes
0,2019-07-02,705.0,CVG,DCA,411,0.0,18:00:00,1.544569e+09,38.0,31.6,...,0.0,Clear,clear-night,"[TT136, 72408013739, 72408594732, 72407493780,...",obs,NaN,7,2,18,0
1,2018-02-24,1724.0,LAS,DEN,628,0.0,21:00:00,1.554080e+09,48.8,44.1,...,0.0,Partially cloudy,partly-cloudy-night,"[KFTY, 72219013874, KATL, 72219503888, 7221965...",obs,NaN,2,24,21,0
4,2019-01-03,1720.0,PHL,ERI,300,0.0,20:00:00,1.544494e+09,45.0,45.0,...,0.0,Partially cloudy,partly-cloudy-night,"[72059400188, 72242953910, KIAH, KDWH, KMCJ, A...",obs,NaN,1,3,20,0
5,2019-06-13,1417.0,MSP,CWA,175,0.0,05:00:00,1.563527e+09,76.9,76.9,...,0.0,Partially cloudy,partly-cloudy-night,"[99999913752, 72215899999, 72406093721, KBWI, ...",obs,NaN,6,13,5,0
6,2018-04-10,2238.0,FLL,BWI,925,44.0,16:00:00,1.562616e+09,81.0,80.8,...,7.0,Partially cloudy,partly-cloudy-day,"[72537094847, 72537614853, KONZ, KYIP, C4874, ...",obs,NaN,4,10,16,0


In [21]:
df.head()

,fl_date,dep_time,origin,dest,distance,weather_delay,datetime,datetimeEpoch,temp,feelslike,...,uvindex,conditions,icon,stations,source,tzoffset,month,day,hours,minutes
0,2019-07-02,705.0,CVG,DCA,411,0.0,18:00:00,1.544569e+09,38.0,31.6,...,0.0,Clear,clear-night,"[TT136, 72408013739, 72408594732, 72407493780,...",obs,NaN,7,2,18,0
1,2018-02-24,1724.0,LAS,DEN,628,0.0,21:00:00,1.554080e+09,48.8,44.1,...,0.0,Partially cloudy,partly-cloudy-night,"[KFTY, 72219013874, KATL, 72219503888, 7221965...",obs,NaN,2,24,21,0
4,2019-01-03,1720.0,PHL,ERI,300,0.0,20:00:00,1.544494e+09,45.0,45.0,...,0.0,Partially cloudy,partly-cloudy-night,"[72059400188, 72242953910, KIAH, KDWH, KMCJ, A...",obs,NaN,1,3,20,0
5,2019-06-13,1417.0,MSP,CWA,175,0.0,05:00:00,1.563527e+09,76.9,76.9,...,0.0,Partially cloudy,partly-cloudy-night,"[99999913752, 72215899999, 72406093721, KBWI, ...",obs,NaN,6,13,5,0
6,2018-04-10,2238.0,FLL,BWI,925,44.0,16:00:00,1.562616e+09,81.0,80.8,...,7.0,Partially cloudy,partly-cloudy-day,"[72537094847, 72537614853, KONZ, KYIP, C4874, ...",obs,NaN,4,10,16,0


In [22]:
#df.to_csv('../data/flights_with_weather_and_articles.csv', index=False)

## Modeling

In [23]:
import xgboost as xgb
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df = df.drop(columns=['fl_date', 'datetime', 'dep_time', 'source', 'datetimeEpoch', 'stations', 'conditions', 'origin', 'dest'])

# Helper function to extract the first element from a list
def first_element(list_like):
    if isinstance(list_like, list) and len(list_like) > 0:
        return list_like[0]
    return list_like

# Apply this function to your categorical columns if they contain lists
for col in ['preciptype', 'icon', 'month', 'day', 'hours']:
    if col in df.columns:
        df[col] = df[col].apply(first_element).astype('category')

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df, columns=['preciptype', 'icon', 'month', 'day', 'hours'])

# Assuming 'weather_delay' is the target variable
X = df_encoded.drop(columns=['weather_delay'])
y = df_encoded['weather_delay']

# Fill NaN values with a placeholder if that makes sense for your data
X.fillna(-999, inplace=True)
y.fillna(0, inplace=True)

# Ensure all data is numeric
X = X.apply(pd.to_numeric)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Convert data to DMatrix (optimized for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set up parameters for XGBoost
params = {
    'max_depth': 4,
    'eta': 0.1,
    'objective': 'reg:squarederror'
}

# Train the model
bst = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions and evaluate
preds = bst.predict(dtest)
rmse = math.sqrt(mean_squared_error(y_test, preds))
print(f"RMSE: {rmse}")

RMSE: 66.42963874587433


In [25]:
import joblib

joblib.dump(bst, '../models/my_xgboost.pkl')

['../models/my_xgboost.pkl']

In [26]:
X.to_csv('../data/training.csv', index=False)
y.to_csv('../data/test.csv', index=False)

In [27]:
bst.get_score(importance_type='weight')

{'distance': 193.0,
 'temp': 67.0,
 'feelslike': 60.0,
 'humidity': 99.0,
 'dew': 64.0,
 'precip': 42.0,
 'snow': 7.0,
 'snowdepth': 15.0,
 'windgust': 26.0,
 'windspeed': 50.0,
 'winddir': 50.0,
 'pressure': 89.0,
 'visibility': 24.0,
 'cloudcover': 64.0,
 'solarradiation': 60.0,
 'solarenergy': 7.0,
 'uvindex': 1.0,
 'icon_clear-night': 7.0,
 'icon_partly-cloudy-night': 5.0,
 'month_1': 15.0,
 'month_2': 6.0,
 'month_3': 13.0,
 'month_4': 3.0,
 'month_5': 3.0,
 'month_6': 2.0,
 'month_7': 17.0,
 'month_8': 13.0,
 'month_9': 4.0,
 'month_10': 11.0,
 'month_11': 4.0,
 'month_12': 3.0,
 'day_1': 5.0,
 'day_2': 6.0,
 'day_3': 1.0,
 'day_4': 8.0,
 'day_5': 4.0,
 'day_6': 4.0,
 'day_7': 13.0,
 'day_8': 3.0,
 'day_9': 6.0,
 'day_10': 2.0,
 'day_11': 5.0,
 'day_12': 5.0,
 'day_13': 14.0,
 'day_15': 10.0,
 'day_16': 4.0,
 'day_17': 34.0,
 'day_18': 4.0,
 'day_19': 23.0,
 'day_20': 19.0,
 'day_21': 16.0,
 'day_22': 1.0,
 'day_23': 2.0,
 'day_24': 7.0,
 'day_25': 2.0,
 'day_26': 3.0,
 'day_27':

In [20]:
preds

array([3.2753568, 4.590068 , 5.457252 , ..., 8.792928 , 0.7766119,
       1.3881005], dtype=float32)